In [2]:
from socketserver import StreamRequestHandler

import pandas as pd
import numpy as np
#Cargar el excel.
df = pd.read_excel("dataset.xlsx")

In [3]:
#TRATAMIENTO DEL DATASET

In [4]:
# Crear columna de fecha unificada
df["FECHA"] = pd.to_datetime(
    dict(year=df["VISITYR"], month=df["VISITMO"], day=df["VISITDAY"]),
    errors='coerce'
)
# Eliminar las columnas originales de año, mes y día
df = df.drop(columns=["VISITYR", "VISITMO", "VISITDAY"])

In [5]:
#Tratamiento de valores faltantes.

In [6]:
#Diccionario con variables faltantes
valores_faltantes = {
    'CDRSUM': [16.5, 17.5],
    'NACCGDS': [-4, 88],
    'HYPERTEN': [-4, 9, 2],
    'DIABETES': [-4, 9, 2],
    'PD': [-4, 9],
    'STROKE': [-4],
    'RACE': [50, 99],
    'EDUC': [99],
}
# Reemplazo esos valores por Nans
for columna, valores in valores_faltantes.items():
    df[columna] = df[columna].replace(valores, np.nan)

print("\nPorcentaje de NaNs por variable:")
porcentaje_nans = df.isna().mean().sort_values(ascending=False) * 100
print(porcentaje_nans)


Porcentaje de NaNs por variable:
STROKE      46.925568
HYPERTEN    40.067023
DIABETES    37.746339
PD          36.974656
NACCGDS     13.692495
NACCAGE      2.679367
RACE         1.632523
EDUC         0.503694
CDRSUM       0.196251
FECHA        0.002562
NACCID       0.001537
SEX          0.001537
dtype: float64


In [7]:
# Número de filas antes de eliminar
print(f"Filas antes de eliminar: {df.shape[0]}")
# Eliminar filas con al menos un NaN en cualquier columna
df = df.dropna()
# Número de filas después de eliminar
print(f"Filas después de eliminar: {df.shape[0]}")

Filas antes de eliminar: 195158
Filas después de eliminar: 83517


In [8]:
# Contar número de visitas por paciente
visitas_por_paciente = df["NACCID"].value_counts()
# Obtener los pacientes con 3 o más visitas
pacientes_validos = visitas_por_paciente[visitas_por_paciente >= 3].index
# Filtrar el DataFrame para quedarte solo con esos pacientes
df = df[df["NACCID"].isin(pacientes_validos)]
# Mostrar número de filas finales
print(f"Filas tras filtrar pacientes con 3 o más visitas: {df.shape[0]}")

Filas tras filtrar pacientes con 3 o más visitas: 61620


In [9]:
# Calcular número de visitas por paciente
# Agrupa por NACCID y cuenta cuántas visitas o filas tiene cada paciente
visitas_por_paciente = df.groupby("NACCID").size()

# Mezcla el orden de los pacientes para seleccionar un subconjunto aleatorio
pacientes_barajados = visitas_por_paciente.sample(frac=1, random_state=42)

# Inicializar lista de pacientes seleccionados y contador de visitas
pacientes_seleccionados = []
total_filas = 0

# Recorrer pacientes barajados y añadirlos hasta llegar a ~40.000 visitas
for paciente, visitas in pacientes_barajados.items():
    if total_filas + visitas > 40000:
        break  # Detener el bucle si superaríamos las 40.000 filas
    pacientes_seleccionados.append(paciente)  # Añadir paciente a la lista
    total_filas += visitas  # Sumar sus visitas al total acumulado

# Filtrar el DataFrame para conservar solo los pacientes seleccionados
df = df[df["NACCID"].isin(pacientes_seleccionados)]

# Mostrar resultado final
print(f"Filas finales tras recorte a 40.000: {df.shape[0]}")
print(f"Pacientes con ≥3 visitas incluidos: {len(pacientes_seleccionados)}")

Filas finales tras recorte a 40.000: 39998
Pacientes con ≥3 visitas incluidos: 8295


In [10]:
#OBJETIVO 1

In [11]:
#Clasica en 3 grupos la variable principal que es CDRSUM.
def clasificar_gravedad(CDRSUM):
    if CDRSUM <= 4.0:
        return "leve"
    elif CDRSUM<= 9.0:
        return "moderada"
    else:
        return "grave"

#Creo la variable que divide en estos 3 estadios
df["GRAVEDAD_CDRSUM"] = df["CDRSUM"].apply(clasificar_gravedad)

In [13]:
# Contar número de filas por grupo
conteo = df["GRAVEDAD_CDRSUM"].value_counts().sort_index()

print("Número de filas (visitas) por grupo de gravedad:")
print(conteo)

Número de filas (visitas) por grupo de gravedad:
GRAVEDAD_CDRSUM
grave        1682
leve        33521
moderada     4795
Name: count, dtype: int64


In [14]:
# Contar pacientes únicos por grupo de gravedad
pacientes_por_grupo = df.groupby("GRAVEDAD_CDRSUM")["NACCID"].nunique()

print("Número de pacientes únicos por grupo de gravedad:")
print(pacientes_por_grupo)


Número de pacientes únicos por grupo de gravedad:
GRAVEDAD_CDRSUM
grave       1000
leve        7410
moderada    2187
Name: NACCID, dtype: int64


In [ ]:
#Como es normal en esta división, aunque no está balanceado, pero es normal porque hay muchos leves
#porque probablemente hayan dejado el seguimiento pronto.